# Imports

In [14]:
import pandas as pd
import numpy as np
import pickle
import pickle

# Constants 

In [15]:
RECIPES_SMALL = "../EDA_files/recipes_small.parquet"
RATINGS_SMALL = "../EDA_files/ratings_small.parquet"
INDEX_TO_RECIPE_OBJ = "../EDA_files/index_to_recipe.obj"
RECIPE_TO_INDEX_OBJ = "../EDA_files/recipe_to_index.obj"
REVIEWS_PER_RECIPE_WITH_0 = "../EDA_files/reviews_per_recipe_with_0.obj"

# Load data

In [16]:
ratings_small = pd.read_parquet(RATINGS_SMALL)
recipes_small = pd.read_parquet(RECIPES_SMALL)
with open(INDEX_TO_RECIPE_OBJ, "rb") as f:
    index_to_recipe = pickle.load(f) 
with open(RECIPE_TO_INDEX_OBJ, "rb") as f:
    recipe_to_index = pickle.load(f) 

In [17]:
with open(REVIEWS_PER_RECIPE_WITH_0, "rb") as f:
    reviews_per_recipe = pickle.load(f)
    
reviews_per_recipe = reviews_per_recipe['ReviewsPerRecipe'].to_dict()

# Random predictions

In [18]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from metrics import predictions_to_tuple_list
from collections import defaultdict


dataset = Dataset.load_from_df(ratings_small[['AuthorId', 'RecipeId', 'Rating']][:100], Reader(rating_scale=(1, 5)))

# Retrieve the trainset.
trainset, testset = train_test_split(dataset, test_size=.25)

# Build an algorithm, and train it.
algo = NormalPredictor()
algo.fit(trainset)
predictions = algo.test(testset)

In [10]:
def predictions_to_dataframe(predictions):
    if (len(predictions) > 0):
        if (type(predictions[0]) != tuple):
             predictions_tuple = predictions_to_tuple_list(predictions)
    pred_df = pd.DataFrame(predictions_tuple, columns=("AuthorId", "RecipeId", "Rating", "PredictedRating", "WasImpossible"))
    pred_df['WasImpossible'] = pred_df['WasImpossible'].apply(lambda x: x['was_impossible'])
    return pred_df

def predictions_to_tuple_list(predictions):
    return [tuple(x) for x in predictions]

In [6]:
def get_top_n(predictions, n=10, min_rating=3.5):
    """
    Creates top n lists for each user. 
    Parameters:
    predictions (list of tuples (uid, iid, real_rating, est_rating)) - predictions made by recommendation algorithm
    n (int) - size of list
    min_rating - minimal predicted rating for item to be used in top_n
    
    Result
    top_n (dictionary) - top n list for each user
    """
    top_n = dict(list)
    for uid, iid, r_ui, est, _ in predictions:
        if (est >= min_rating):
            top_n[uid].append((iid, r_ui, est))
    for uid, ratings in top_n.items():
        ratings.sort(key=lambda x: x[2], reverse=True)
        top_n[uid] = ratings[:n]
    return top_n

In [8]:
def get_relevant_items_for_user(uid, ratings_df, min_rating=3.5):
    '''
    Creates list of items user alreardy liked
    Parameters:
    uid - user id
    ratings_df - dataframe with user-item rankings
    min_rating - minimal rating to be included 
    
    '''
    return ratings_df[(ratings_df['AuthorId']==uid) & (ratings_df['Rating']>=min_rating)]["RecipeId"].tolist()

In [27]:
def get_relevant_items(predictions, ratings_df, min_ratings=3.5):
    '''
    Creates list of relevant items for each user
    Parameterers:
    predictions (list of tuples (uid, iid, real_rating, est_rating)) - predictions made by recommendation algorithm
    ratings_df - dataframe with user-item rankings
    min_rating - minimal rating to be included
    
    Result
    Dictionary with list of relevant items for each user
    '''
    relevant_items = defaultdict(list)
    unique_users = set([uid for uid, iid, r_ui, est, _ in predictions])
    for uid in unique_users:
        relevant_items[uid] = get_relevant_items_for_user(uid, ratings_df)
    return relevant_items

In [13]:
def map_predictions_to_user(predictions):
    '''
    Creates list of predicted items for each user
    '''
    predictions_per_user = defaultdict(list)
    for uid, iid, r_ui, est, _ in predictions:
        predictions_per_user[uid].append((iid, r_ui, est))
    return predictions_per_user

In [28]:
def precision_per_user(recommendations_per_user: list, relevant_items_per_user: list):
    '''
    Returns precision for one user
    
    Parameters
    recommendations_per_user (list) - list of predictions for one user
    relevant_items_per_user - list of items from user's history
    
    Result
    precision - Precision for one user
    '''
    recommended_and_relevant_items = [item for item in recommendations_per_user if item in relevant_items_per_user]
    precision = len(recommended_and_relevant_items) / len(recommendations_per_user) if len(recommendations_per_user) != 0 else 0
    return precision

In [29]:
def recommender_precision_at_k(recommendations: dict, relevant_items: dict, k=10):
    '''
    Calculates precisions across all users
    
    Parameters:
    recommendations (dict) - dictionary of recommendations for all users
    relevant_items (dict) - dictionary of relevant items for all users
    
    Result:
    precisions (dict) - precision for all users
    '''
    precisions = dict()
    for uid in recommendations.keys():
        if len(recommendations[uid]) > k:
            recommended_items_for_user = recommendations[uid][:k]
        else:
            recommended_items_for_user = recommendations[uid]
        
        precisions[uid] = precision_per_user(recommended_items_for_user, relevant_items[uid])
    return precisions

In [30]:
def recall_per_user(recommendations_per_user: list, relevant_items_per_user: list):
    '''
    Returns recall for one user
    
    Parameters
    recommendations_per_user (list) - list of predictions for one user
    relevant_items_per_user - list of items from user's history
    
    Result
    recall - recall for one user
    '''
    recommended_and_relevant_items = [item for item in recommendations_per_user if item in relevant_items_per_user]
    recall = len(recommended_and_relevant_items) / len(relevant_items_per_user) if len(relevant_items_per_user) != 0 else 0
    return recall

In [31]:
def recommender_recall_at_k(recommendations: dict, relevant_items: dict):
    '''
    Calculates recalls across all users
    
    Parameters:
    recommendations (dict) - dictionary of recommendations for all users
    relevant_items (dict) - dictionary of relevant items for all users
    
    Result:
    recalls (dict) - precision for all users
    '''
    recalls = dict()
    for uid in recommendations.keys():
        recalls[uid] = recall_per_user(recommendations[uid], relevant_items[uid])
    return recalls

In [32]:
def average_precision_at_k(recommendations_per_user: list, relevant_items_per_user: list, k=10):
    '''
    Calculates avarage precision@k for one user
    Parameters:
    recommendations_per_user (list) - list of predictions for one user
    relevant_items_per_user - list of items from user's history

    Result
    apk - average precision at k for user
    '''
    if len(relevant_items_per_user) == 0:
        return 0.0
    
    if len(recommendations_per_user) > k:
            recommendations_per_user = recommendations_per_user[:k]
    hits = 0.0
    precision_sum = 0.0
    for i, item in enumerate(recommendations_per_user):
        if item in relevant_items_per_user:
            hits += 1.0
            precision_sum += hits / (i + 1.0)
    apk = precision_sum / min(len(recommendation_per_user), k)
    return apk

In [33]:
def recommender_map(recommendations: dict, relevant_items: dict):
    '''
    Calculates mean average precision for recommender system at k
    
    Parameters:
    recommendations (dict) - dictionary of recommendations for all users
    relevant_items (dict) - dictionary of relevant items for all users
    
    Result:
    MAP@k
    '''
    apks = []
    for uid in recommendations.keys():
        apks.append(average_precision_at_k(recommendations[uid], relvant_items[uid]))
    return np.mean(apks)

In [34]:
def hit_rate(recommendations: dict, left_out_predictions: list):
    '''
    Calculates hit rate for all users - number of users with at least one good prediction in top_n divided by number of all users 
    
    Parameters:
    recommendations - dictionary of predicted items for each user
    left_out_predictions -   list of all ratings left in the test set
    
    Result:
    hit rate
    '''
    hits = 0
    total = 0
    for (left_out_user, left_out_item, rating) in left_out_predictions:
        hit = False
        for item, _, est_rating in recommendations[uid]:
            if (left_out_item == item):
                hit = True
                break
        if(hit):
            hits += 1
    return hits / len(left_out_predictions)

In [21]:
def ARHR(recommendations: dict, left_out_predictions: list):
    '''
    Calculates average reciprocal hit rank - similar to hit rate, but takes into consideration rank in the recommended list
    
    Parameters:
    recommendations - dictionary of predicted items for each user
    left_out_predictions -   list of all ratings left in the test set
    
    Result:
    arhr
    '''
    hits = 0
    for (left_out_user, left_out_item, rating) in left_out_predictions:
        rank = 0
        hit_rank = 0
        for item, _, _ in recommendations[uid]:
            rank += 1
            if (left_out_item == item):
                hit_rank = rank
                break
        if(hit_rank > 0):
            hits += 1.0 / hit_rank
    return hits / len(left_out_predictions)

In [22]:
def user_coverage(top_n: dict, number_of_users: int, min_rating=3.5):
    '''
    What percent of users have 1 good prediction
    
    Parameters
    top_n - dictionary of predicted items for each user
    number_of_users - number of all users in the system
    min_rating - minimum rating for item to be considered
    
    '''
    hits = 0
    for uid in top_n.keys():
        hit = False
        for item, _, est  in top_n[uid]:
            if (est > min_rating):
                hit = True
                break
        if(hit):
            hits += 1
    return hits / number_of_users

In [23]:
def item_coverage(top_n: dict, number_of_items: int, min_rating=3.5):
    '''
    Calculates what percantage of items' catalog can be recommended by algorithm
    
    Parameters:
    top_n - dictionary of predicted items for each user
    number_of_users - number of all users in the system
    min_rating - minimum rating for item to be considered
    '''
    items = []
    for value in top_n.values():
        for pairs in value:
            items.append(pairs[0])
    unique_items = set(items)
    return len(unique_items) / number_of_items

In [25]:
def novelty(top_n, ranking):
    '''
    Calculates average popularity of recommended items in top_n lists across all users
    
    Parameters:
    top_n (dict) - dictionary of predicted items for each user
    ranking (dict) - popularity rank for each item
    '''
    n = 0
    total = 0
    for uid in top_n.keys():
        for rating in top_n[uid]:
            item_id = rating[0]
            rank = ranking[item_id]
            total += rank
            n += 1
    return total / n

In [26]:
def diversity(top_n, item_vectors):
    '''
    Calculates diversity of recommended items in top_n lists
    
    Parameters:
    top_n (dict) - dictionary of predicted items for each user
    item_vectors - 
    '''
    n = 0
    total = 0
    for uid in top_n.keys():
        pairs = itertools.combinations(top_n[uid], 2)
        for pair in pairs:
            item1 = pair[0][0]
            item2 = pair[1][0]
            # compute similarity between item1 & item2
            # similarity = 
            # total += similarity
            # n += 1
    total_similarity = total / n
    diversity = 1 - total_similarity
    return diversity

In [6]:
import math
def DCG(top_n: dict, relevant_items: dict):
    ''' 
    Calculates discounted cumulative gain
    
    Parameters:
    top_n (dict) - dictionary of predicted items for each user
    relevant_items (dict) - dictionary of relevant items for all users
    '''
    def relevance_function(is_relevant):
        return 2**int(is_relevant) - 1
    score = 0
    for user, items in top_n.items():
        rank = 0
        for item in items:
            is_relevant = is_item_relevant_for_user(item, user, relevant_items)
            score += relevance_function(is_relevant) / math.log2(rank + 1)
            rank += 1
    return score / len(top_n.keys())

In [24]:
def get_popularity_ranks(reviews_per_recipe):
    rankings = defaultdict(int)
    rank = 1
    for item_id, count in sorted(reviews_per_recipe.items(), key=lambda x: x[1], reverse=True):
        rankings[item_id] = rank
        rank += 1
    return rankings

In [1]:
def is_item_relevant_for_user(item, user, relevant_items: dict):
    return item in relevant_items[uid]

In [14]:
def precision_recall_at_k(predictions, relevant_items, top_n, k=10, min_rating=3.5):
    '''
    
    '''
    predictions_per_user = map_predictions_to_user(predictions)
    precisions = dict()
    recalls = dict()
    for uid, user_ratings in predictions_per_user.items():
        relevant_items_per_user = relevant_items[uid]
        try:
            recommended_items_per_user, _, _ = zip(*top_n[uid])
        except:
            recommended_items_per_user = []
        recommended_and_relevant_items = list(set(relevant_items_per_user) & set(recommended_items_per_user))
        
        precisions[uid] = len(recommended_and_relevant_items) / len(recommended_items_per_user) if len(recommended_items_per_user) != 0 else 0
        recalls[uid] = len(recommended_and_relevant_items) / len(relevant_items_per_user) if len(relevant_items_per_user) != 0 else 0
    return precisions, recalls